In [2]:
import nltk

In [3]:
# читаем файл
rawData = open("SMSSpamCollection.tsv", encoding='utf8').read()

In [4]:
rawData[:500]

"ham\tI've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.\nspam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\nham\tNah I don't think he goes to usf, he lives around here though\nham\tEven my brother is not like to speak with me. They treat me like aid"

In [5]:
parsedData = rawData.replace('\t', '\n').split('\n')

In [6]:
parsedData

['ham',
 "I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.",
 'spam',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'ham',
 "Nah I don't think he goes to usf, he lives around here though",
 'ham',
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 'ham',
 'I HAVE A DATE ON SUNDAY WITH WILL!!',
 'ham',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'spam',
 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'spam',
 'Had your mobile 11 months or more? U R entitled to Upda

In [7]:
# в отдельный массив выведем метки спам не спам и сами тексты. Все четные строки это флаг на спамность нечетные 
labelList = parsedData[0::2]
textList = parsedData[1::2]

In [8]:
print(labelList[0:5])
print(textList[0:5])

['ham', 'spam', 'ham', 'ham', 'ham']
["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.", "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", "Nah I don't think he goes to usf, he lives around here though", 'Even my brother is not like to speak with me. They treat me like aids patent.', 'I HAVE A DATE ON SUNDAY WITH WILL!!']


In [9]:
import pandas as pd
fullCorpus = pd.DataFrame({'label': labelList[:-1], 'body_list':textList})

In [10]:
fullCorpus

,label,body_list
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
...,...,...
5565,spam,This is the 2nd time we have tried 2 contact u...
5566,ham,Will ü b going to esplanade fr home?
5567,ham,"Pity, * was in mood for that. So...any other s..."
5568,ham,The guy did some bitching but I acted like i'd...


In [11]:
fullCorpus = pd.read_csv("SMSSpamCollection.tsv", sep="\t", header=None)

In [12]:
fullCorpus.columns= ['label', 'body_text']

In [13]:
fullCorpus

,label,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...
5564,ham,Will ü b going to esplanade fr home?
5565,ham,"Pity, * was in mood for that. So...any other s..."
5566,ham,The guy did some bitching but I acted like i'd...


In [14]:
fullCorpus.groupby('label').count()

,body_text
label,
ham,4822
spam,746


In [15]:
fullCorpus.loc[:,['label']].isnull().sum()

label    0
dtype: int64

In [16]:
fullCorpus.loc[:,['body_text']].isnull().sum()

body_text    0
dtype: int64

In [17]:
import pymorphy2

In [40]:
# Попробовал pyMorphy но он здесь не подойдет т.к. язык английский для английского будем использовать nltk.PorterStemmer
import re
morph = pymorphy2.MorphAnalyzer()
def normalizePymorphy(text):
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        words.append(pv[0].normal_form + '_' + str(pv[0].tag.POS)) # Берем наиболее вероятную форму.
    return words   

In [41]:
fullCorpus['norm_text'] = fullCorpus.body_text.apply(normalizePymorphy)

In [42]:
fullCorpus

,label,body_text,norm_text,stemmer_text,body_text_clean,body_text_clean_token,body_text_nostop
0,ham,I've been searching for the right words to tha...,"[i_None, ve_None, been_None, searching_None, f...","[i, ve, been, search, for, the, right, word, t...",Ive been searching for the right words to than...,"[ive, been, searching, for, the, right, words,...","[ive, searching, right, words, thank, breather..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free_None, entry_None, in_None, a_None, wkly_...","[free, entri, in, a, wkli, comp, to, win, fa, ...",Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah_None, i_None, don_None, t_None, think_Non...","[nah, i, don, t, think, he, goe, to, usf, he, ...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t..."
3,ham,Even my brother is not like to speak with me. ...,"[even_None, my_None, brother_None, is_None, no...","[even, my, brother, is, not, like, to, speak, ...",Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[i_None, have_None, a_None, date_None, on_None...","[i, have, a, date, on, sunday, with, will]",I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[date, sunday]"
...,...,...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[this_None, is_None, the_None, nd_None, time_N...","[thi, is, the, nd, time, we, have, tri, contac...",This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, 750, poun..."
5564,ham,Will ü b going to esplanade fr home?,"[will_None, b_None, going_None, to_None, espla...","[will, b, go, to, esplanad, fr, home]",Will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity_None, was_None, in_None, mood_None, for_...","[piti, wa, in, mood, for, that, so, ani, other...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]"
5566,ham,The guy did some bitching but I acted like i'd...,"[the_None, guy_None, did_None, some_None, bitc...","[the, guy, did, some, bitch, but, i, act, like...",The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b..."


In [21]:
#поскольку pymorphy только для русского языка для английского возьмем PorterStemmer

In [43]:
from nltk.stem import PorterStemmer

In [44]:
porter = PorterStemmer()
def normalizePorterStemmer(text):
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        pv = porter.stem(t.lower())
        words.append(pv) # Берем наиболее вероятную форму.
    return words   

In [45]:
fullCorpus['stemmer_text'] = fullCorpus.body_text.apply(normalizePorterStemmer)

In [46]:
# для того чтобы стеммер нормально заработал сначала очистим от хлама текст
fullCorpus

,label,body_text,norm_text,stemmer_text,body_text_clean,body_text_clean_token,body_text_nostop
0,ham,I've been searching for the right words to tha...,"[i_None, ve_None, been_None, searching_None, f...","[i, ve, been, search, for, the, right, word, t...",Ive been searching for the right words to than...,"[ive, been, searching, for, the, right, words,...","[ive, searching, right, words, thank, breather..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free_None, entry_None, in_None, a_None, wkly_...","[free, entri, in, a, wkli, comp, to, win, fa, ...",Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah_None, i_None, don_None, t_None, think_Non...","[nah, i, don, t, think, he, goe, to, usf, he, ...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t..."
3,ham,Even my brother is not like to speak with me. ...,"[even_None, my_None, brother_None, is_None, no...","[even, my, brother, is, not, like, to, speak, ...",Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[i_None, have_None, a_None, date_None, on_None...","[i, have, a, date, on, sunday, with, will]",I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[date, sunday]"
...,...,...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[this_None, is_None, the_None, nd_None, time_N...","[thi, is, the, nd, time, we, have, tri, contac...",This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, 750, poun..."
5564,ham,Will ü b going to esplanade fr home?,"[will_None, b_None, going_None, to_None, espla...","[will, b, go, to, esplanad, fr, home]",Will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity_None, was_None, in_None, mood_None, for_...","[piti, wa, in, mood, for, that, so, ani, other...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]"
5566,ham,The guy did some bitching but I acted like i'd...,"[the_None, guy_None, did_None, some_None, bitc...","[the, guy, did, some, bitch, but, i, act, like...",The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b..."


In [47]:
# удаляем пунктуацию
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [48]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct


In [49]:
fullCorpus['body_text_clean'] = fullCorpus.body_text.apply(lambda x: remove_punct(x))

In [50]:
fullCorpus

,label,body_text,norm_text,stemmer_text,body_text_clean,body_text_clean_token,body_text_nostop
0,ham,I've been searching for the right words to tha...,"[i_None, ve_None, been_None, searching_None, f...","[i, ve, been, search, for, the, right, word, t...",Ive been searching for the right words to than...,"[ive, been, searching, for, the, right, words,...","[ive, searching, right, words, thank, breather..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free_None, entry_None, in_None, a_None, wkly_...","[free, entri, in, a, wkli, comp, to, win, fa, ...",Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah_None, i_None, don_None, t_None, think_Non...","[nah, i, don, t, think, he, goe, to, usf, he, ...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t..."
3,ham,Even my brother is not like to speak with me. ...,"[even_None, my_None, brother_None, is_None, no...","[even, my, brother, is, not, like, to, speak, ...",Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[i_None, have_None, a_None, date_None, on_None...","[i, have, a, date, on, sunday, with, will]",I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[date, sunday]"
...,...,...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[this_None, is_None, the_None, nd_None, time_N...","[thi, is, the, nd, time, we, have, tri, contac...",This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, 750, poun..."
5564,ham,Will ü b going to esplanade fr home?,"[will_None, b_None, going_None, to_None, espla...","[will, b, go, to, esplanad, fr, home]",Will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity_None, was_None, in_None, mood_None, for_...","[piti, wa, in, mood, for, that, so, ani, other...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]"
5566,ham,The guy did some bitching but I acted like i'd...,"[the_None, guy_None, did_None, some_None, bitc...","[the, guy, did, some, bitch, but, i, act, like...",The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b..."


In [30]:
# проводим токенизацию

In [31]:
import re
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens



In [51]:
fullCorpus['body_text_tokenized'] = fullCorpus.body_text_clean.apply(lambda x: tokenize(x.lower()))

In [52]:
fullCorpus

,label,body_text,norm_text,stemmer_text,body_text_clean,body_text_clean_token,body_text_nostop,body_text_tokenized
0,ham,I've been searching for the right words to tha...,"[i_None, ve_None, been_None, searching_None, f...","[i, ve, been, search, for, the, right, word, t...",Ive been searching for the right words to than...,"[ive, been, searching, for, the, right, words,...","[ive, searching, right, words, thank, breather...","[ive, been, searching, for, the, right, words,..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free_None, entry_None, in_None, a_None, wkly_...","[free, entri, in, a, wkli, comp, to, win, fa, ...",Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, in, 2, a, wkly, comp, to, win, f..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah_None, i_None, don_None, t_None, think_Non...","[nah, i, don, t, think, he, goe, to, usf, he, ...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, i, dont, think, he, goes, to, usf, he, l..."
3,ham,Even my brother is not like to speak with me. ...,"[even_None, my_None, brother_None, is_None, no...","[even, my, brother, is, not, like, to, speak, ...",Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids...","[even, my, brother, is, not, like, to, speak, ..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[i_None, have_None, a_None, date_None, on_None...","[i, have, a, date, on, sunday, with, will]",I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[date, sunday]","[i, have, a, date, on, sunday, with, will]"
...,...,...,...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[this_None, is_None, the_None, nd_None, time_N...","[thi, is, the, nd, time, we, have, tri, contac...",This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, 750, poun...","[this, is, the, 2nd, time, we, have, tried, 2,..."
5564,ham,Will ü b going to esplanade fr home?,"[will_None, b_None, going_None, to_None, espla...","[will, b, go, to, esplanad, fr, home]",Will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]","[will, ü, b, going, to, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity_None, was_None, in_None, mood_None, for_...","[piti, wa, in, mood, for, that, so, ani, other...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]","[pity, was, in, mood, for, that, soany, other,..."
5566,ham,The guy did some bitching but I acted like i'd...,"[the_None, guy_None, did_None, some_None, bitc...","[the, guy, did, some, bitch, but, i, act, like...",The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b...","[the, guy, did, some, bitching, but, i, acted,..."


In [34]:
#remove_stopwords
import nltk
stopword = nltk.corpus.stopwords.words('english')

In [35]:
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [56]:
# функция для удалили стоп-слова
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

In [57]:
fullCorpus['body_text_nostop'] = fullCorpus['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

In [58]:
fullCorpus

,label,body_text,norm_text,stemmer_text,body_text_clean,body_text_clean_token,body_text_nostop,body_text_tokenized
0,ham,I've been searching for the right words to tha...,"[i_None, ve_None, been_None, searching_None, f...","[i, ve, been, search, for, the, right, word, t...",Ive been searching for the right words to than...,"[ive, been, searching, for, the, right, words,...","[ive, searching, right, words, thank, breather...","[ive, been, searching, for, the, right, words,..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free_None, entry_None, in_None, a_None, wkly_...","[free, entri, in, a, wkli, comp, to, win, fa, ...",Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, in, 2, a, wkly, comp, to, win, f..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah_None, i_None, don_None, t_None, think_Non...","[nah, i, don, t, think, he, goe, to, usf, he, ...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, i, dont, think, he, goes, to, usf, he, l..."
3,ham,Even my brother is not like to speak with me. ...,"[even_None, my_None, brother_None, is_None, no...","[even, my, brother, is, not, like, to, speak, ...",Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids...","[even, my, brother, is, not, like, to, speak, ..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[i_None, have_None, a_None, date_None, on_None...","[i, have, a, date, on, sunday, with, will]",I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[date, sunday]","[i, have, a, date, on, sunday, with, will]"
...,...,...,...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[this_None, is_None, the_None, nd_None, time_N...","[thi, is, the, nd, time, we, have, tri, contac...",This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, 750, poun...","[this, is, the, 2nd, time, we, have, tried, 2,..."
5564,ham,Will ü b going to esplanade fr home?,"[will_None, b_None, going_None, to_None, espla...","[will, b, go, to, esplanad, fr, home]",Will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]","[will, ü, b, going, to, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity_None, was_None, in_None, mood_None, for_...","[piti, wa, in, mood, for, that, so, ani, other...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]","[pity, was, in, mood, for, that, soany, other,..."
5566,ham,The guy did some bitching but I acted like i'd...,"[the_None, guy_None, did_None, some_None, bitc...","[the, guy, did, some, bitch, but, i, act, like...",The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b...","[the, guy, did, some, bitching, but, i, acted,..."


In [64]:
# Пишем одну функцию, которая соединяет в себе удаление пунктуации токенизацию и удаление стоп-слов. Загргузим заново dataframe

data = pd.read_csv("SMSSpamCollection.tsv", sep="\t", header=None)
data.columns= ['label', 'body_text']

In [98]:
import string
string.punctuation
import nltk
stopwords = nltk.corpus.stopwords.words('english')
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text


data['body_text_nostop'] = data['body_text'].apply(lambda x: clean_text(x.lower()))

In [99]:
data

,label,body_text,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,ham,I've been searching for the right words to tha...,"[ive, searching, right, words, thank, breather...","[ive, search, right, word, thank, breather, pr...","[ive, searching, right, word, thank, breather,..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...","[nah, dont, think, go, usf, life, around, though]"
3,ham,Even my brother is not like to speak with me. ...,"[even, brother, like, speak, treat, like, aids...","[even, brother, like, speak, treat, like, aid,...","[even, brother, like, speak, treat, like, aid,..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[date, sunday]","[date, sunday]","[date, sunday]"
...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, 750, poun...","[2nd, time, tri, 2, contact, u, u, 750, pound,...","[2nd, time, tried, 2, contact, u, u, 750, poun..."
5564,ham,Will ü b going to esplanade fr home?,"[ü, b, going, esplanade, fr, home]","[ü, b, go, esplanad, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity, mood, soany, suggestions]","[piti, mood, soani, suggest]","[pity, mood, soany, suggestion]"
5566,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, id, interested, b...","[guy, bitch, act, like, id, interest, buy, som...","[guy, bitching, acted, like, id, interested, b..."


In [100]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
# стеммингуем
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

data['body_text_stemmed'] = data['body_text_nostop'].apply(lambda x: stemming(x))

In [101]:
data

,label,body_text,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,ham,I've been searching for the right words to tha...,"[ive, searching, right, words, thank, breather...","[ive, search, right, word, thank, breather, pr...","[ive, searching, right, word, thank, breather,..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...","[nah, dont, think, go, usf, life, around, though]"
3,ham,Even my brother is not like to speak with me. ...,"[even, brother, like, speak, treat, like, aids...","[even, brother, like, speak, treat, like, aid,...","[even, brother, like, speak, treat, like, aid,..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[date, sunday]","[date, sunday]","[date, sunday]"
...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, 750, poun...","[2nd, time, tri, 2, contact, u, u, 750, pound,...","[2nd, time, tried, 2, contact, u, u, 750, poun..."
5564,ham,Will ü b going to esplanade fr home?,"[ü, b, going, esplanade, fr, home]","[ü, b, go, esplanad, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity, mood, soany, suggestions]","[piti, mood, soani, suggest]","[pity, mood, soany, suggestion]"
5566,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, id, interested, b...","[guy, bitch, act, like, id, interest, buy, som...","[guy, bitching, acted, like, id, interested, b..."


In [102]:
# Сделаем лемматизацию
import nltk
wn = nltk.WordNetLemmatizer()

In [103]:
# Пишем функцию лемматизации
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

In [104]:
data['body_text_lemmatized'] = data['body_text_nostop'].apply(lambda x: lemmatizing(x))

In [105]:
data

,label,body_text,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,ham,I've been searching for the right words to tha...,"[ive, searching, right, words, thank, breather...","[ive, search, right, word, thank, breather, pr...","[ive, searching, right, word, thank, breather,..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...","[nah, dont, think, go, usf, life, around, though]"
3,ham,Even my brother is not like to speak with me. ...,"[even, brother, like, speak, treat, like, aids...","[even, brother, like, speak, treat, like, aid,...","[even, brother, like, speak, treat, like, aid,..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[date, sunday]","[date, sunday]","[date, sunday]"
...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, 750, poun...","[2nd, time, tri, 2, contact, u, u, 750, pound,...","[2nd, time, tried, 2, contact, u, u, 750, poun..."
5564,ham,Will ü b going to esplanade fr home?,"[ü, b, going, esplanade, fr, home]","[ü, b, go, esplanad, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...","[pity, mood, soany, suggestions]","[piti, mood, soani, suggest]","[pity, mood, soany, suggestion]"
5566,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, id, interested, b...","[guy, bitch, act, like, id, interest, buy, som...","[guy, bitching, acted, like, id, interested, b..."


In [106]:
# Проводим векторизацию

In [107]:
from sklearn.feature_extraction.text import CountVectorizer

In [108]:
count_vect = CountVectorizer(analyzer=clean_text)

In [109]:
X_counts = count_vect.fit(data['body_text'])

In [110]:
count_vect.get_feature_names()

['',
 '0',
 '008704050406',
 '0089my',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '020603',
 '0207',
 '02070836089',
 '02072069400',
 '02073162414',
 '02085076972',
 '020903',
 '021',
 '050703',
 '0578',
 '06',
 '060505',
 '061104',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '071104',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07786200117',
 '077xxx',
 '078',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '0784987',
 '0789xxxxxxx',
 '0794674629107880867867',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '0825',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '08450542832',
 '08452810071',
 '0

In [111]:
data_sample = data[:20]

In [112]:
data_sample

,label,body_text,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,ham,I've been searching for the right words to tha...,"[ive, searching, right, words, thank, breather...","[ive, search, right, word, thank, breather, pr...","[ive, searching, right, word, thank, breather,..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...","[nah, dont, think, go, usf, life, around, though]"
3,ham,Even my brother is not like to speak with me. ...,"[even, brother, like, speak, treat, like, aids...","[even, brother, like, speak, treat, like, aid,...","[even, brother, like, speak, treat, like, aid,..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[date, sunday]","[date, sunday]","[date, sunday]"
5,ham,As per your request 'Melle Melle (Oru Minnamin...,"[per, request, melle, melle, oru, minnaminungi...","[per, request, mell, mell, oru, minnaminungint...","[per, request, melle, melle, oru, minnaminungi..."
6,spam,WINNER!! As a valued network customer you have...,"[winner, valued, network, customer, selected, ...","[winner, valu, network, custom, select, receiv...","[winner, valued, network, customer, selected, ..."
7,spam,Had your mobile 11 months or more? U R entitle...,"[mobile, 11, months, u, r, entitled, update, l...","[mobil, 11, month, u, r, entitl, updat, latest...","[mobile, 11, month, u, r, entitled, update, la..."
8,ham,I'm gonna be home soon and i don't want to tal...,"[im, gonna, home, soon, dont, want, talk, stuf...","[im, gonna, home, soon, dont, want, talk, stuf...","[im, gonna, home, soon, dont, want, talk, stuf..."
9,spam,"SIX chances to win CASH! From 100 to 20,000 po...","[six, chances, win, cash, 100, 20000, pounds, ...","[six, chanc, win, cash, 100, 20000, pound, txt...","[six, chance, win, cash, 100, 20000, pound, tx..."


In [113]:
count_vect_sample = CountVectorizer(analyzer=clean_text)
X_counts_sample = count_vect_sample.fit_transform(data_sample['body_text'])

In [114]:
X_counts_sample.shape

(20, 204)

In [121]:
count_vect_sample.get_feature_names()

['08002986030',
 '08452810075over18s',
 '09061701461',
 '1',
 '100',
 '100000',
 '11',
 '12',
 '150pday',
 '16',
 '2',
 '20000',
 '2005',
 '21st',
 '3',
 '4',
 '4403ldnw1a7rw18',
 '4txtú120',
 '6days',
 '81010',
 '87077',
 '87121',
 '87575',
 '9',
 '900',
 'aft',
 'aids',
 'already',
 'anymore',
 'apply',
 'ard',
 'around',
 'b',
 'blessing',
 'breather',
 'brother',
 'call',
 'callers',
 'callertune',
 'camera',
 'cash',
 'chances',
 'claim',
 'click',
 'co',
 'code',
 'colour',
 'comin',
 'comp',
 'copy',
 'cost',
 'credit',
 'cried',
 'csh11',
 'cup',
 'customer',
 'da',
 'date',
 'dont',
 'eg',
 'eh',
 'england',
 'enough',
 'entitled',
 'entry',
 'even',
 'fa',
 'feel',
 'final',
 'fine',
 'finish',
 'first',
 'free',
 'friends',
 'fulfil',
 'go',
 'goalsteam',
 'goes',
 'going',
 'gonna',
 'gota',
 'granted',
 'ha',
 'help',
 'hl',
 'home',
 'hours',
 'httpwap',
 'im',
 'info',
 'ive',
 'jackpot',
 'joking',
 'k',
 'kim',
 'kl341',
 'lar',
 'latest',
 'lccltd',
 'like',
 'link',


In [125]:
X_counts_df = pd.DataFrame(X_counts_sample.toarray())

In [126]:
# теперь вместо имен столбцов вставим сами токены
X_counts_df.columns = count_vect_sample.get_feature_names()

In [127]:
X_counts_df

,08002986030,08452810075over18s,09061701461,1,100,100000,11,12,150pday,16,...,wkly,wonderful,wont,word,words,wwwdbuknet,xxxmobilemovieclub,xxxmobilemovieclubcomnqjkgighjjgcbl,yes,ü
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [117]:
# Теперь тоже самое на всем датафрейме
count_vect = CountVectorizer(analyzer=clean_text)
X_counts = count_vect.fit_transform(data['body_text'])

In [119]:
X_counts.shape

(5568, 9443)

In [120]:
count_vect.get_feature_names()

['',
 '0',
 '008704050406',
 '0089my',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '020603',
 '0207',
 '02070836089',
 '02072069400',
 '02073162414',
 '02085076972',
 '020903',
 '021',
 '050703',
 '0578',
 '06',
 '060505',
 '061104',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '071104',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07786200117',
 '077xxx',
 '078',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '0784987',
 '0789xxxxxxx',
 '0794674629107880867867',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '0825',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '08450542832',
 '08452810071',
 '0